In [1]:
from db_schema import engine, Incident, Category, Participant
import pandas as pd
from datetime import datetime as dt
from sqlalchemy.orm import sessionmaker
from us_state_abbrev import us_state_abbrev as abr
import time

In [2]:
Session = sessionmaker(bind=engine)

In [3]:
session = Session()

In [4]:
csv_data = pd.read_csv('data/data-clean.csv', parse_dates=['date'])
csv_data['incident_characteristics'] = csv_data['incident_characteristics'].fillna('')
csv_data['participant_age'] = csv_data['participant_age'].fillna('')
csv_data['participant_gender'] = csv_data['participant_gender'].fillna('').str.lower()
csv_data['participant_status'] = csv_data['participant_status'].fillna('').str.lower()
csv_data['participant_type'] = csv_data['participant_type'].fillna('').str.lower()
csv_data.head()

,incident_id,date,state,n_killed,n_injured,incident_characteristics,participant_age,participant_gender,participant_status,participant_type
0,461105,2013-01-01,Pennsylvania,0,4,Shot - Wounded/Injured||Mass Shooting (4+ vict...,0::20,0::male||1::male||3::male||4::female,0::arrested||1::injured||2::injured||3::injure...,0::victim||1::victim||2::victim||3::victim||4:...
1,460726,2013-01-01,California,1,3,"Shot - Wounded/Injured||Shot - Dead (murder, a...",0::20,0::male,0::killed||1::injured||2::injured||3::injured,0::victim||1::victim||2::victim||3::victim||4:...
2,478855,2013-01-01,Ohio,1,3,"Shot - Wounded/Injured||Shot - Dead (murder, a...",0::25||1::31||2::33||3::34||4::33,0::male||1::male||2::male||3::male||4::male,"0::injured, unharmed, arrested||1::unharmed, a...",0::subject-suspect||1::subject-suspect||2::vic...
3,478925,2013-01-05,Colorado,4,0,"Shot - Dead (murder, accidental, suicide)||Off...",0::29||1::33||2::56||3::33,0::female||1::male||2::male||3::male,0::killed||1::killed||2::killed||3::killed,0::victim||1::victim||2::victim||3::subject-su...
4,478959,2013-01-07,North Carolina,2,2,"Shot - Wounded/Injured||Shot - Dead (murder, a...",0::18||1::46||2::14||3::47,0::female||1::male||2::male||3::female,0::injured||1::injured||2::killed||3::killed,0::victim||1::victim||2::victim||3::subject-su...


In [5]:
all_categories = {}
incidents = []
now = time.time()
for _,row in csv_data.iterrows():
    if _ > 0 and _ % 10000 == 0:
        print(f'{_} records processed')
    incident = Incident(date=row['date'],
                       state=abr[row['state']],
                       n_killed=row['n_killed'],
                       n_injured=row['n_injured'])
    # Categories
    categories = [cat for cat in row['incident_characteristics'].split('|') if cat]
    for cat in categories:
        category = all_categories.get(cat)
        if not category:
            category = Category(name=cat)
            all_categories[cat] = category
        incident.categories.append(category)
    # Participants
    participants = {}
    # - Age
    parts = [p for p in row['participant_age'].split('|') if p]
    for part in parts:
        if not part:
            continue
        i,raw_age = [p for p in part.split(':') if p]
        age = int(raw_age) if raw_age.isdigit() else None
        participant = participants.get(i)
        if not participant:
            participant = Participant(age=age)
            incident.participants.append(participant)
            participants[i] = participant
    # - Gender
    parts = [p for p in row['participant_gender'].split('|') if p]
    for part in parts:
        if not part:
            continue
        i,raw_gender = [p for p in part.split(':') if p]
        is_male=True if raw_gender == 'male' else False if raw_gender == 'female' else None
        participant = participants.get(i)
        if not participant:
            participant = Participant(is_male=is_male)
            incident.participants.append(participant)
            participants[i] = participant
        else:
            participant.is_male = is_male
    # - Status
    parts = [p for p in row['participant_status'].split('|') if p]
    for part in parts:
        if not part:
            continue
        i,raw_status = [p for p in part.split(':') if p]
        is_killed = True if 'killed' in raw_status else False if 'injured' in raw_status else None
        participant = participants.get(i)
        if not participant:
            participant = Participant(is_killed=is_killed)
            incident.participants.append(participant)
            participants[i] = participant
        else:
            participant.is_killed = is_killed
    # - Type
    parts = [p for p in row['participant_type'].split('|') if p]
    for part in parts:
        if not part:
            continue
        i,raw_type = [p for p in part.split(':') if p]
        is_victim = True if 'victim' in raw_type else \
                    False if 'suspect' in raw_type or 'subject' in raw_type else None
        participant = participants.get(i)
        if not participant:
            participant = Participant(is_victim=is_victim)
            incident.participants.append(participant)
            participants[i] = participant
        else:
            participant.is_victim = is_victim
    incidents.append(incident)
print('Created all incidents. Commiting...')
session.add_all(incidents)
session.commit()
now = int(time.time() - now)
print(f'Importing {len(csv_data)} incidents took {now // 60}:{now % 60:02d}')

10000 records processed
20000 records processed
30000 records processed
40000 records processed
50000 records processed
60000 records processed
70000 records processed
80000 records processed
90000 records processed
100000 records processed
110000 records processed
120000 records processed
130000 records processed
140000 records processed
150000 records processed
160000 records processed
170000 records processed
180000 records processed
190000 records processed
200000 records processed
210000 records processed
220000 records processed
230000 records processed
Created all incidents. Commiting...
Importing 239677 incidents took 2:56


In [6]:
#!jupyter nbconvert --to Script db_import